In [6]:
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
#from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np
from pydub import AudioSegment
from scipy.io import wavfile
from tempfile import mktemp

from datasets import load_dataset

In [21]:
ds = load_dataset("mozilla-foundation/common_voice_11_0", "en", split='train[:100]')

Found cached dataset common_voice_11_0 (/home/hetricke/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/en/11.0.0/3f27acf10f303eac5b6fbbbe02495aeddb46ecffdb0a2fe3507fcfbf89094631)


In [23]:
def transcript_prep(transcription):
    if transcription.startswith('"') and transcription.endswith('"'):
        # we can remove trailing quotation marks as they do not affect the transcription
        transcription = transcription[1:-1]

    if len(transcription) > 0:
        if transcription[-1] not in [".", "?", "!"]:
            # append a full-stop to sentences that do not end in punctuation
            transcription = transcription + "."

    return transcription


In [24]:
def audio_to_spectrogram(audio_file):
    mp3_audio = AudioSegment.from_file(audio_file, format="mp3")  # read mp3
    wname = mktemp('.wav')  # use temporary file
    mp3_audio.export(wname, format="wav")  # convert to wav
    FS, data = wavfile.read(wname)  # read wav file

    file_name = audio_file[:-4] + ".png"

    plt.specgram(data, Fs=FS, NFFT=128, noverlap=0)  # plot
    plt.axis('off')
    plt.savefig(file_name, bbox_inches='tight')
    return file_name

In [25]:

def prepare_dataset(batch):
  """Function to preprocess the dataset with the .map method"""
  batch["sentence"] = transcript_prep(batch["sentence"])
  batch["path"] = audio_to_spectrogram(batch["path"])

  return batch



In [26]:
ds = ds.map(prepare_dataset, desc="preprocess dataset")

preprocess dataset:   0%|          | 0/100 [00:00<?, ? examples/s]/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)
preprocess dataset:   1%|          | 1/100 [00:00<00:19,  5.21 examples/s]/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)
preprocess dataset:   2%|▏         | 2/100 [00:00<00:22,  4.30 examples/s]/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)
preprocess dataset:   3%|▎         | 3/100 [00:00<00:28,  3.46 examples/s]/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero encountered in l

In [ ]:
split = ds.train_test_split(test_size = 0.2,train_size = 0.8)
train = split['train']
test = split['test']

x_train = train["path"]
y_train = train["sentence"]

x_test = test["path"]
y_test = test["sentence"]


In [ ]:
#build model
model = keras.Sequential()
model.add(layers.Dense(units = 128, activation = 'relu'))
model.add(layers.Dense(units = 128, activation='relu'))
model.add(layers.Dense(units = 128, activation ='relu'))
model.add(layers.SimpleRNN(units = 128, activation = 'relu'))
model.add(layers.Dense(29, activation = 'softmax'))

In [ ]:
#train model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size= 100)


In [ ]:
#add computer i/o for speech recognition